In [ ]:
import sys 
sys.path.append(rf'C:\Users\simof\Documents\GitHub\thesis\data_processing')
from importlib import reload
import utils
reload(utils)
from utils import *

In [ ]:
confini = gpd.read_file('data/clean_data/confini_comunali/confini_comunali.shp')
centroidi = gpd.read_file('data/clean_data/centroidi_comunali/centroidi_comunali.shp')
comuni_piemontesi = pd.read_csv('data/clean_data/elenco_comuni_piemonte.csv')
confini.info()

In [ ]:
confini[confini['comune'] == 'none']
confini.loc[confini[confini['comune'] == 'none'].index, 'comune'] = 'nonetorinese'
# cities with letters with stresses and other signs
check_strange_letters(confini, 'comune')

Two municipalities from Val d'Aosta, they don't matter

In [ ]:
confini_piemonte = confini[confini['comune'].isin(comuni_piemontesi['Comune'])]
confini_piemonte.info()

In [ ]:
print(confini_piemonte['comune'].duplicated().sum())
print(confini_piemonte['geometry'].duplicated().sum())
print(confini_piemonte[['comune', 'geometry']].duplicated().sum())

In [ ]:
i = confini_piemonte.index
i1 = confini_piemonte[confini_piemonte['comune'].duplicated()].index
i2 = confini_piemonte[confini_piemonte[['comune', 'geometry']].duplicated()].index
groups = confini_piemonte[confini_piemonte['comune'].isin(confini_piemonte.loc[i[i.isin(i1) & (~i.isin(i2))], :]['comune'])].groupby('comune')

for id, group in groups:
    print(id)
    group.plot()
    plt.show()

In [ ]:
# first fix the other "regular" duplicates
no_dup_confini_piemonte = confini_piemonte.drop_duplicates(['comune', 'geometry'])
no_dup_confini_piemonte['comune'].duplicated().sum()

There's another city in Trentino Alto Adige named Samone, I can delete this one

In [ ]:
mask = no_dup_confini_piemonte['comune'] == 'samone'
no_dup_confini_piemonte[mask].centroid

In [ ]:
no_dup_confini_piemonte.drop(3719, axis=0, inplace=True)
no_dup_confini_piemonte.plot();

Almost there

In [ ]:
confini_piemonte2 = no_dup_confini_piemonte[no_dup_confini_piemonte['comune'].isin(comuni_piemontesi['Comune'])]
confini_piemonte2.info()

In [ ]:
confini_piemonte2.plot();

Only 20 municipalities left

In [ ]:
save_clean_data(confini_piemonte2, 'confini_piemonte')

---

In [ ]:
confini = gpd.read_file('data/clean_data/confini_piemonte/confini_piemonte.shp')
comuni = pd.read_csv('data/clean_data/elenco_comuni_piemonte.csv')
save_clean_data(confini.merge(comuni.rename({'Comune': 'comune'}, axis=1), on='comune', how='inner'
                              ).loc[:, ['cartodb_id', 'comune', 'Codice Comune', 'geometry']], 'confini_piemonte')

---

Input missing cities manually

In [ ]:
comuni = gpd.read_file('data/clean_data/confini_comunali/confini_comunali.shp')
piemonte = gpd.read_file('data/clean_data/confini_piemonte/confini_piemonte.shp')
display(comuni.head())
display(piemonte.head())

In [ ]:
comuni.loc[[840, 8900], 'comune'] = 'nonetorinese'
mask1 = comuni['comune'].isin(piemonte['comune'])
mask2 = (comuni.bounds.iloc[:, 0] >= piemonte.total_bounds[0]) & (comuni.bounds.iloc[:, 1] >= piemonte.total_bounds[1]) & (comuni.bounds.iloc[:, 2] <= piemonte.total_bounds[2]) &(comuni.bounds.iloc[:, 3] <= piemonte.total_bounds[3])
difference = comuni[~mask1 & mask2]
print(difference)

In [ ]:
print(difference['comune'].duplicated().sum())
print(difference[['comune', 'geometry']].duplicated().sum())

In [ ]:
comuni_quasi_mancanti = difference.drop_duplicates()

fig, ax = plt.subplots(1,2)

comuni_quasi_mancanti.plot(ax=ax[0])
piemonte.plot(ax=ax[1])

plt.show()

In [ ]:
import networkx as nx

graph = nx.Graph()
for idx, poly in zip(difference.index, difference.geometry):
    graph.add_node(idx)
    for other_idx, other_poly in zip(difference.index, difference.geometry):
        if idx != other_idx and poly.touches(other_poly):
            graph.add_edge(idx, other_idx)

components = list(nx.connected_components(graph))

In [ ]:
sorted_components = sorted(components, key=len, reverse=True)
lombardia = graph.subgraph(sorted_components[0])
liguria = graph.subgraph(sorted_components[1])
aosta = graph.subgraph(sorted_components[2])

regions = list(sorted_components[0])
regions.extend(list(sorted_components[1]))
regions.extend(list(sorted_components[2]))

no_regions_difference = difference.drop(regions, axis=0)
print(no_regions_difference)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10,6))

piemonte.plot(ax=ax, color='blue', alpha=0.5)
comuni_quasi_mancanti.plot(ax=ax, color='red', alpha=0.5)
no_regions_difference.plot(ax=ax, color='green', alpha=0.5)

plt.tight_layout()
plt.show()

In [ ]:
pezzo_nord = comuni_quasi_mancanti[comuni_quasi_mancanti['comune'].isin(['rivavaldobbia', 'rimasangiuseppe', 'rimasco'])]
pezzo_ovest = comuni_quasi_mancanti[comuni_quasi_mancanti['comune'].isin(['vicocanavese', 'trausella', 'meugliano', 'lugnacco', 'alicesuperiore', 'pecco'])]
pezzo_fuori = comuni_quasi_mancanti[comuni_quasi_mancanti['comune'].isin(["campioned'italia", 'armo'])]

piemonte_completo = pd.concat([piemonte, no_regions_difference, pezzo_nord, pezzo_ovest], ignore_index=True)
piemonte_completo.drop(piemonte_completo[piemonte_completo['comune'].isin(["campioned'italia", 'armo'])].index, axis=0, inplace=True)
piemonte_completo.plot();

In [ ]:
graph2 = nx.Graph()
for idx, poly in zip(piemonte_completo.index, piemonte_completo.geometry):
    graph2.add_node(idx)
    for other_idx, other_poly in zip(piemonte_completo.index, piemonte_completo.geometry):
        if idx != other_idx and poly.touches(other_poly):
            graph2.add_edge(idx, other_idx)

components2 = list(nx.connected_components(graph2))

print(piemonte_completo.duplicated('comune').sum())
print(piemonte_completo.duplicated('geometry').sum())
print(piemonte_completo.duplicated(['comune', 'geometry']).sum())

In [ ]:
piemonte2 = piemonte_completo.drop_duplicates(['geometry', 'comune'])
piemonte2[piemonte2.duplicated('geometry', keep=False)]

piemonte2[piemonte2['comune'] == 'calliano']
piemonte2.loc[1183, 'comune'] = 'callianomonferrato'

elenco = pd.read_csv('data/clean_data/elenco_comuni.csv')
li = piemonte2[piemonte2['Codice Com'].isna()]['comune']
print(elenco[elenco['Comune'].isin(li)])
print(piemonte2[piemonte2['Codice Com'].isna()]['comune'])

In [ ]:
di = {}
di['soprana'] = 'A096062'
di['gavazzana'] = 'A006080'
di['cursolo-orasso'] = 'A103027'
di['moransengo'] = 'A005079'
di['piovamassaia'] = 'A005086'
di['pont-canavese'] = 'A001199'
di['fubine'] = 'A006076'
di['montemagno'] = 'A005077'
di['crosa'] = 'A096022'
di['sabbia'] = 'A002123'
di['cellio'] = 'A002038'
di['cavaglio-spoccia'] = 'A103020'
di['quaregna'] = 'A096051'
di['alluvionicambio'] = 'A006006'
di['veruno'] = 'A003157'
di['castellania'] = 'A006045'
di['casorzo'] = 'A005020'
di['selvemarcone'] = 'A096061'
di['grana'] = 'A005056'
di['lu'] = 'A006089'
di['viganella'] = 'A103073'
di['falmenta'] = 'A103030'
di['callianomonferrato'] = 'A005014'
di['mosso'] = 'A096084'
di['trivero'] = 'A096070'
di['camo'] = 'A004036'
di['vallemosso'] = 'A096073'
di['quittengo'] = 'A096052'
di['valmala'] = 'A004236'
di['sanpaolocervo'] = 'A096060'
di['piovera'] = 'A006130'
di['breia'] = 'A002019'
di['castellar'] = 'A004048'
di['cerretocastello'] = 'A096017'
di['gattico'] = 'A003166'
di['seppiana'] = 'A103063'
di['tonengo'] = 'A005110'
di['cuccaromonferrato'] = 'A006064'
di['rimasco'] = 'A002112'
di['rimasangiuseppe'] = 'A002111'
di['rivavaldobbia'] = 'A002114'
di['pecco'] = 'A001182'
di['meugliano'] = 'A001151'
di['alicesuperiore'] = 'A001005'
di['vicocanavese'] = 'A001297'
di['lugnacco'] = 'A001138'
di['trausella'] = 'A001277'

print(pd.Series(di.keys())[~pd.Series(di.keys()).isin(piemonte2[piemonte2['Codice Com'].isna()]['comune'])])

In [ ]:
def impose_code(row, dic):
    if row['comune'] in dic.keys():
        row['Codice Com'] = di[row['comune']]
    return row

piemonte3 = piemonte2.apply(lambda x: impose_code(x, di), axis=1)
piemonte3.info()

In [ ]:
print(piemonte3.comune.duplicated().sum())
print(piemonte3['Codice Com'].duplicated().sum())
print(piemonte3[['comune', 'Codice Com']].duplicated().sum())
print(piemonte3.geometry.duplicated().sum())

In [ ]:
save_clean_data(piemonte3, 'confini_piemonte')

---